In [5]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
import random

In [43]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

In [8]:
import joblib

## Constantes (paths, dimensões, etc)

In [9]:
base_dir_train_crack = 'dataset/train/crack/'
base_dir_train_non_crack = 'dataset/train/non-crack_random_subset/'
base_dir_test_crack = 'dataset/test/crack/'
base_dir_test_non_crack = 'dataset/test/non-crack_random_subset/'
base_dir_processed_train_images = 'processed-dataset-train/'

In [10]:
preprocessed_image_name_prefix = 'preprocessed_'

In [11]:
default_image_width = 256
default_image_height = 256

In [12]:
median_blur_kernel_size = 3
clahe_clip_limit = 4.0
clahe_tile_grid_size = (30, 30)
first_erosion_iterations = 3
first_dilation_iterations = 2
size_for_erosion_and_dilation_element = 2

element_for_erosion_and_dilation = \
    cv2.getStructuringElement(cv2.MORPH_CROSS, \
                              (2 * size_for_erosion_and_dilation_element + 1, 2 * size_for_erosion_and_dilation_element + 1), \
                              (size_for_erosion_and_dilation_element, size_for_erosion_and_dilation_element))

adapt_thresh_neighbour_size = 11
adapt_thresh_subtraction_constant = 2

sobel_kernel_size = 3

In [13]:
svm_param_grid_for_search = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

## Funções de pré-processamento

In [14]:
def pre_process_with_grayscale_median_clahe_darkening_erosion_dilation( \
                                                                 images_list, \
                                                                 median_blur_kernel_size, \
                                                                 clahe_clip_limit, \
                                                                 clahe_tile_grid_size, \
                                                                 element_for_erosion_and_dilation, \
                                                                 first_erosion_iterations, \
                                                                 first_dilation_iterations):
    
    pre_processed_images = []
    clahe = cv2.createCLAHE(clipLimit = clahe_clip_limit, tileGridSize = clahe_tile_grid_size)
    
    for image in images_list:
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # converte para escala de cinzas
        blured_image = cv2.medianBlur(grayscale_image, median_blur_kernel_size) # aplica filtro de mediana
        clahe_image = clahe.apply(blured_image) # aplica equalizacao de histograma por blocos, uniformiza iluminacao
        min_pixel_value = np.min(clahe_image) # obtem o pixel de menor valor na imagem
        darker_image = clahe_image - min_pixel_value # escurece toda a imagem, subtraindo o valor do menor pixel
        
        erosion_image = cv2.erode(darker_image, \
                                  element_for_erosion_and_dilation, \
                                  iterations = first_erosion_iterations) # aplica n erosoes
        
        dilated_image = cv2.dilate(erosion_image, \
                                   element_for_erosion_and_dilation, \
                                   iterations = first_dilation_iterations) # aplica n dilatacoes
        
        pre_processed_images.append(dilated_image)
        
    return pre_processed_images

In [48]:
def pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_adaptthreshold(
                                                                            images_list, \
                                                                            median_blur_kernel_size, \
                                                                            clahe_clip_limit, \
                                                                            clahe_tile_grid_size, \
                                                                            element_for_erosion_and_dilation, \
                                                                            first_erosion_iterations, \
                                                                            first_dilation_iterations, \
                                                                            adapt_thresh_neighbour_size, \
                                                                            adapt_thresh_subtraction_constant):
    
    pre_processed_images = []
    clahe = cv2.createCLAHE(clipLimit = clahe_clip_limit, tileGridSize = clahe_tile_grid_size)
    
    for image in images_list:
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # converte para escala de cinzas
        blured_image = cv2.medianBlur(grayscale_image, median_blur_kernel_size) # aplica filtro de mediana
        clahe_image = clahe.apply(blured_image) # aplica equalizacao de histograma por blocos, uniformiza iluminacao
        min_pixel_value = np.min(clahe_image) # obtem o pixel de menor valor na imagem
        darker_image = clahe_image - min_pixel_value # escurece toda a imagem, subtraindo o valor do menor pixel
        
        erosion_image = cv2.erode(darker_image, \
                                  element_for_erosion_and_dilation, \
                                  iterations = first_erosion_iterations) # aplica n erosoes
        
        dilated_image = cv2.dilate(erosion_image, \
                                   element_for_erosion_and_dilation, \
                                   iterations = first_dilation_iterations) # aplica n dilatacoes
        
        image_threshold = cv2.adaptiveThreshold(dilated_image, 255, \
                                                cv2.ADAPTIVE_THRESH_MEAN_C, \
                                                cv2.THRESH_BINARY, \
                                                adapt_thresh_neighbour_size, \
                                                adapt_thresh_subtraction_constant) # binarização por threshold adaptativo
        
        pre_processed_images.append(image_threshold)
        
    return pre_processed_images

In [16]:
def pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_sobel(
                                                                   images_list, \
                                                                   median_blur_kernel_size, \
                                                                   clahe_clip_limit, \
                                                                   clahe_tile_grid_size, \
                                                                   element_for_erosion_and_dilation, \
                                                                   first_erosion_iterations, \
                                                                   first_dilation_iterations, \
                                                                   sobel_kernel_size):
    
    pre_processed_images = []
    clahe = cv2.createCLAHE(clipLimit = clahe_clip_limit, tileGridSize = clahe_tile_grid_size)
    
    for image in images_list:
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # converte para escala de cinzas
        blured_image = cv2.medianBlur(grayscale_image, median_blur_kernel_size) # aplica filtro de mediana
        clahe_image = clahe.apply(blured_image) # aplica equalizacao de histograma por blocos, uniformiza iluminacao
        min_pixel_value = np.min(clahe_image) # obtem o pixel de menor valor na imagem
        darker_image = clahe_image - min_pixel_value # escurece toda a imagem, subtraindo o valor do menor pixel
        
        erosion_image = cv2.erode(darker_image, \
                                  element_for_erosion_and_dilation, \
                                  iterations = first_erosion_iterations) # aplica 2 erosoes
        
        dilated_image = cv2.dilate(erosion_image, \
                                   element_for_erosion_and_dilation, \
                                   iterations = first_dilation_iterations) # aplica 2 dilatacoes
        
        gradient_x = cv2.Sobel(dilated_image, cv2.CV_64F, 1, 0, ksize=sobel_kernel_size)
        gradient_y = cv2.Sobel(dilated_image, cv2.CV_64F, 0, 1, ksize=sobel_kernel_size)
        gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
        gradient_magnitude = cv2.convertScaleAbs(gradient_magnitude) # obtém o filtro sobel horizontal e vertical
        
        pre_processed_images.append(gradient_magnitude)
        
    return pre_processed_images

## Demais funções

In [24]:
def load_images_from_name_list(base_dir, list_with_names, default_image_width, default_image_height):
    loaded_images = []
    for image_name in list_with_names:
        im = cv2.imread(base_dir + image_name)
        im_correct_colorscheme = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im_resized = cv2.resize(im_correct_colorscheme, (default_image_width, default_image_height))
        loaded_images.append(im_resized)
    return loaded_images

In [18]:
def save_images(base_dir, images_list, images_names_list, image_name_prefix, directory_to_save):
    for idx in range(len(images_names_list)):
        
        if not os.path.exists(base_dir + directory_to_save):
            os.makedirs(base_dir + directory_to_save)
            
        cv2.imwrite(base_dir + directory_to_save + '/' + image_name_prefix + images_names_list[idx], images_list[idx])

In [19]:
def get_data_and_labels(pre_processed_train_crack, pre_processed_train_non_crack):
    input_masks = []
    input_masks.extend(pre_processed_train_crack)
    input_masks.extend(pre_processed_train_non_crack) # adicionamos todas as imagens à lista de input do modelo
    input_labels = []
    input_labels.extend(np.ones((len(pre_processed_train_crack), ), np.uint8))
    input_labels.extend(np.zeros((len(pre_processed_train_non_crack), ), np.uint8)) # adicionamos as labels das imagens

    input_masks = list(map(lambda x:x.flatten(), input_masks)) # aqui, redimensionamos as imagens para terem 1 dimensao

    zipped_list_for_shuffle = list(zip(input_masks, input_labels))
    random.shuffle(zipped_list_for_shuffle)
    
    shuffled_input_masks, shuffled_input_labels = zip(*zipped_list_for_shuffle)
    
    return list(shuffled_input_masks), list(shuffled_input_labels)

## Carregando conjunto de dados

In [20]:
crack_images_for_train_name_list = os.listdir(base_dir_train_crack)
non_crack_images_for_train_name_list = os.listdir(base_dir_train_non_crack)

crack_images_for_test_name_list = os.listdir(base_dir_test_crack)
non_crack_images_for_test_name_list = os.listdir(base_dir_test_non_crack)

In [25]:
crack_images_for_train_list = load_images_from_name_list(base_dir_train_crack, \
                                                         crack_images_for_train_name_list, \
                                                         default_image_width, default_image_height)

In [26]:
non_crack_images_for_train_list = load_images_from_name_list(base_dir_train_non_crack, \
                                                             non_crack_images_for_train_name_list, \
                                                             default_image_width, default_image_height)

In [30]:
crack_images_for_test_list = load_images_from_name_list(base_dir_test_crack, \
                                                         crack_images_for_test_name_list, \
                                                         default_image_width, default_image_height)

In [31]:
non_crack_images_for_test_list = load_images_from_name_list(base_dir_test_non_crack, \
                                                             non_crack_images_for_test_name_list, \
                                                             default_image_width, default_image_height)

## Treinamento dos modelos

### Pré-processamento 1: Conversão escala de cinzas, filtro mediana, CLAHE, escurecimento, erosão e dilatação

In [32]:
pre_processed_train_crack_1 = pre_process_with_grayscale_median_clahe_darkening_erosion_dilation( \
                                                                                       crack_images_for_train_list, \
                                                                                       median_blur_kernel_size, \
                                                                                       clahe_clip_limit, \
                                                                                       clahe_tile_grid_size, \
                                                                                       element_for_erosion_and_dilation, \
                                                                                       first_erosion_iterations, \
                                                                                       first_dilation_iterations)

In [33]:
pre_processed_train_non_crack_1 = pre_process_with_grayscale_median_clahe_darkening_erosion_dilation( \
                                                                                           non_crack_images_for_train_list, \
                                                                                           median_blur_kernel_size, \
                                                                                           clahe_clip_limit, \
                                                                                           clahe_tile_grid_size, \
                                                                                           element_for_erosion_and_dilation, \
                                                                                           first_erosion_iterations, \
                                                                                           first_dilation_iterations)

In [34]:
pre_processed_test_crack_1 = pre_process_with_grayscale_median_clahe_darkening_erosion_dilation( \
                                                                                       crack_images_for_test_list, \
                                                                                       median_blur_kernel_size, \
                                                                                       clahe_clip_limit, \
                                                                                       clahe_tile_grid_size, \
                                                                                       element_for_erosion_and_dilation, \
                                                                                       first_erosion_iterations, \
                                                                                       first_dilation_iterations)

In [35]:
pre_processed_test_non_crack_1 = pre_process_with_grayscale_median_clahe_darkening_erosion_dilation( \
                                                                                           non_crack_images_for_test_list, \
                                                                                           median_blur_kernel_size, \
                                                                                           clahe_clip_limit, \
                                                                                           clahe_tile_grid_size, \
                                                                                           element_for_erosion_and_dilation, \
                                                                                           first_erosion_iterations, \
                                                                                           first_dilation_iterations)

In [36]:
save_images(base_dir_processed_train_images, \
            pre_processed_train_crack_1, \
            crack_images_for_train_name_list, \
            preprocessed_image_name_prefix, \
            'crack_1')

In [37]:
save_images(base_dir_processed_train_images, \
            pre_processed_train_non_crack_1, \
            non_crack_images_for_train_name_list, \
            preprocessed_image_name_prefix, \
            'non-crack_1')

#### SVM

In [38]:
svm_input_data_1, svm_input_labels_1 = get_data_and_labels(pre_processed_train_crack_1, pre_processed_train_non_crack_1)

In [39]:
svm_model_1 = SVC()
grid_search_svm_1 = GridSearchCV(svm_model_1, svm_param_grid_for_search, cv=5, scoring='accuracy')
grid_search_svm_1.fit(svm_input_data_1, svm_input_labels_1)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy')

In [40]:
svm_test_data_1, svm_test_labels_1 = get_data_and_labels(pre_processed_test_crack_1, pre_processed_test_non_crack_1)

In [41]:
print("Best Hyperparameters SVM 1:", grid_search_svm_1.best_params_)
print("Best Accuracy SVM 1:", grid_search_svm_1.best_score_)
best_svm_model_1 = grid_search_svm_1.best_estimator_
accuracy_1 = best_svm_model_1.score(svm_test_data_1, svm_test_labels_1)
print("Test Set Accuracy SVM 1:", accuracy_1)

Best Hyperparameters SVM 1: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Best Accuracy SVM 1: 0.8333333333333333
Test Set Accuracy SVM 1: 0.8407643312101911


In [42]:
svm_test_prediction_1 = best_svm_model_1.predict(svm_test_data_1)

In [45]:
accuracy_svm_1 = accuracy_score(svm_test_prediction_1, svm_test_labels_1)
precision_svm_1 = precision_score(svm_test_labels_1, svm_test_prediction_1)
recall_svm_1 = recall_score(svm_test_labels_1, svm_test_prediction_1)
f1_svm_1 = f1_score(svm_test_prediction_1, svm_test_labels_1)

In [ ]:
print("Precision:", precision_svm_1)
print("Recall:", recall_svm_1)
print("F1-SCORE:", f1_svm_1)

### Pré-processamento 2: Conversão escala de cinzas, filtro mediana, CLAHE, escurecimento, erosão, dilatação e threshold adaptativo

In [49]:
pre_processed_train_crack_2 = \
    pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_adaptthreshold( \
                                                                            crack_images_for_train_list, \
                                                                            median_blur_kernel_size, \
                                                                            clahe_clip_limit, \
                                                                            clahe_tile_grid_size, \
                                                                            element_for_erosion_and_dilation, \
                                                                            first_erosion_iterations, \
                                                                            first_dilation_iterations, \
                                                                            adapt_thresh_neighbour_size, \
                                                                            adapt_thresh_subtraction_constant)

In [50]:
pre_processed_train_non_crack_2 = \
    pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_adaptthreshold( \
                                                                            non_crack_images_for_train_list, \
                                                                            median_blur_kernel_size, \
                                                                            clahe_clip_limit, \
                                                                            clahe_tile_grid_size, \
                                                                            element_for_erosion_and_dilation, \
                                                                            first_erosion_iterations, \
                                                                            first_dilation_iterations, \
                                                                            adapt_thresh_neighbour_size, \
                                                                            adapt_thresh_subtraction_constant)

In [51]:
pre_processed_test_crack_2 = pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_adaptthreshold( \
                                                                            crack_images_for_test_list, \
                                                                            median_blur_kernel_size, \
                                                                            clahe_clip_limit, \
                                                                            clahe_tile_grid_size, \
                                                                            element_for_erosion_and_dilation, \
                                                                            first_erosion_iterations, \
                                                                            first_dilation_iterations, \
                                                                            adapt_thresh_neighbour_size, \
                                                                            adapt_thresh_subtraction_constant)

In [52]:
pre_processed_test_non_crack_2 = \
    pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_adaptthreshold( \
                                                                            non_crack_images_for_test_list, \
                                                                            median_blur_kernel_size, \
                                                                            clahe_clip_limit, \
                                                                            clahe_tile_grid_size, \
                                                                            element_for_erosion_and_dilation, \
                                                                            first_erosion_iterations, \
                                                                            first_dilation_iterations, \
                                                                            adapt_thresh_neighbour_size, \
                                                                            adapt_thresh_subtraction_constant)

In [53]:
save_images(base_dir_processed_train_images, \
            pre_processed_train_crack_2, \
            crack_images_for_train_name_list, \
            preprocessed_image_name_prefix, \
            'crack_2')

In [54]:
save_images(base_dir_processed_train_images, \
            pre_processed_train_non_crack_2, \
            non_crack_images_for_train_name_list, \
            preprocessed_image_name_prefix, \
            'non-crack_2')

Obtendo dados e labels

In [55]:
input_data_2, input_labels_2 = get_data_and_labels(pre_processed_train_crack_2, pre_processed_train_non_crack_2)

In [56]:
test_data_2, test_labels_2 = get_data_and_labels(pre_processed_test_crack_2, pre_processed_test_non_crack_2)

#### SVM

In [ ]:
svm_model_2 = SVC()
grid_search_svm_2 = GridSearchCV(svm_model_2, svm_param_grid_for_search, cv=5, scoring='accuracy')
grid_search_svm_2.fit(input_data_2, input_labels_2)

In [ ]:
print("Best Hyperparameters SVM 2:", grid_search_svm_2.best_params_)
print("Best Accuracy SVM 2:", grid_search_svm_2.best_score_)
best_svm_model_2 = grid_search_svm_2.best_estimator_
accuracy_2 = best_svm_model_2.score(test_data_2, test_labels_2)
print("Test Set Accuracy SVM 2:", accuracy_2)

In [ ]:
svm_test_prediction_2 = best_svm_model_2.predict(test_data_2)

In [ ]:
precision_svm_2 = precision_score(test_labels_2, svm_test_prediction_2)
recall_svm_2 = recall_score(test_labels_2, svm_test_prediction_2)
f1_svm_2 = f1_score(svm_test_prediction_2, test_labels_2)

In [ ]:
print("Precision:", precision_svm_2)
print("Recall:", recall_svm_2)
print("F1-SCORE:", f1_svm_2)

### Pré-processamento 3: Conversão escala de cinzas, filtro mediana, CLAHE, escurecimento, erosão, dilatação e filtro Sobel

In [ ]:
pre_processed_train_crack_3 = \
    pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_sobel( \
                                                                   crack_images_for_train_list, \
                                                                   median_blur_kernel_size, \
                                                                   clahe_clip_limit, \
                                                                   clahe_tile_grid_size, \
                                                                   element_for_erosion_and_dilation, \
                                                                   first_erosion_iterations, \
                                                                   first_dilation_iterations, \
                                                                   sobel_kernel_size)

In [ ]:
pre_processed_train_non_crack_3 = \
    pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_sobel( \
                                                                   non_crack_images_for_train_list, \
                                                                   median_blur_kernel_size, \
                                                                   clahe_clip_limit, \
                                                                   clahe_tile_grid_size, \
                                                                   element_for_erosion_and_dilation, \
                                                                   first_erosion_iterations, \
                                                                   first_dilation_iterations, \
                                                                   sobel_kernel_size)

In [ ]:
pre_processed_test_crack_3 = \
    pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_sobel( \
                                                                   crack_images_for_test_list, \
                                                                   median_blur_kernel_size, \
                                                                   clahe_clip_limit, \
                                                                   clahe_tile_grid_size, \
                                                                   element_for_erosion_and_dilation, \
                                                                   first_erosion_iterations, \
                                                                   first_dilation_iterations, \
                                                                   sobel_kernel_size)

In [ ]:
pre_processed_test_non_crack_3 = \
    pre_process_with_grayscale_median_clahe_darkening_erosion_dilation_sobel( \
                                                                   non_crack_images_for_test_list, \
                                                                   median_blur_kernel_size, \
                                                                   clahe_clip_limit, \
                                                                   clahe_tile_grid_size, \
                                                                   element_for_erosion_and_dilation, \
                                                                   first_erosion_iterations, \
                                                                   first_dilation_iterations, \
                                                                   sobel_kernel_size)

In [ ]:
save_images(base_dir_processed_train_images, \
            pre_processed_train_crack_3, \
            crack_images_for_train_name_list, \
            preprocessed_image_name_prefix, \
            'crack_3')

In [ ]:
save_images(base_dir_processed_train_images, \
            pre_processed_train_non_crack_3, \
            non_crack_images_for_train_name_list, \
            preprocessed_image_name_prefix, \
            'non-crack_3')

Obtendo dados e labels

In [ ]:
input_data_3, input_labels_3 = get_data_and_labels(pre_processed_train_crack_3, pre_processed_train_non_crack_3)

In [ ]:
test_data_3, test_labels_3 = get_data_and_labels(pre_processed_test_crack_3, pre_processed_test_non_crack_3)

#### SVM

In [ ]:
svm_model_3 = SVC()
grid_search_svm_3 = GridSearchCV(svm_model_3, svm_param_grid_for_search, cv=5, scoring='accuracy')
grid_search_svm_3.fit(input_data_3, input_labels_3)

In [ ]:
print("Best Hyperparameters SVM 3:", grid_search_svm_3.best_params_)
print("Best Accuracy SVM 3:", grid_search_svm_3.best_score_)
best_svm_model_3 = grid_search_svm_3.best_estimator_
accuracy_3 = best_svm_model_3.score(test_data_3, test_labels_3)
print("Test Set Accuracy SVM 3:", accuracy_3)

In [ ]:
svm_test_prediction_3 = best_svm_model_3.predict(test_data_3)

In [ ]:
precision_svm_3 = precision_score(test_labels_3, svm_test_prediction_3)
recall_svm_3 = recall_score(test_labels_3, svm_test_prediction_3)
f1_svm_3 = f1_score(svm_test_prediction_3, test_labels_3)

In [ ]:
print("Precision:", precision_svm_3)
print("Recall:", recall_svm_3)
print("F1-SCORE:", f1_svm_3)